### Uncertainty analysis
Using the Monte Carlo Simulations method

In [1]:
import numpy as np
import rasterio as rio

In [2]:
# Load the rasters
ksat = rio.open('Rasters/ksat.tif')
wcs = rio.open('Rasters/wcs.tif')
alt_dev = rio.open('Rasters/alt-dev.tif')

In [3]:
# Convert rasters to numpy arrays
ksat_data = ksat.read()
wcs_data = wcs.read()
alt_dev_data = alt_dev.read()


In [4]:
def monte_carlo_simulation(model, data, num_simulations):


SyntaxError: incomplete input (2009632123.py, line 1)

In [ ]:
monte_carlo_simulation(smca_model.smca_model, smca_data, 10)

# Save the results to a file
with open('simulation_results.txt', 'w') as f:
    for result in monte_carlo_simulation(smca_model.smca_model, smca_data, 10):
        f.write(f"{result}\n")

TypeError: 'module' object is not callable. Did you mean: 'smca_model.smca_model(...)'?

In [4]:
weights = [0.43, 0.14, 0.43]

# Define the type for each criterion
# '1' for benefit, '0' for cost
types = [1, 1, 0]

In [ ]:
def smca_model(ksat_data, wcs_data, alt_dev_data, weights, types):
    # Check if data are the same size/shape, if not raise an error
    if ksat_data.shape != wcs_data.shape or ksat_data.shape != alt_dev_data.shape:
        raise ValueError("Input data must have the same shape")

    # Multiply ksat by the appropriate weight and invert if required
    ksat_weighted = ksat_data * weights[0]
    if types[0] == 0:
        ksat_weighted = 1 - ksat_weighted

    # Multiply wcs by the appropriate weight and invert if required
    wcs_weighted = wcs_data * weights[1]
    if types[1] == 0:
        wcs_weighted = 1 - wcs_weighted

    # Multiply alt_dev by the appropriate weight and invert if required
    alt_dev_weighted = alt_dev_data * weights[2]
    if types[2] == 1:
        alt_dev_weighted = 1 - alt_dev_weighted

    # Sum the weighted data
    result = np.sum([ksat_weighted, wcs_weighted, alt_dev_weighted], axis=0)

    return result
    
smca_model(
    ksat_data,
    wcs_data,
    alt_dev_data,
    weights,
    types
)

# Save the result to a file
profile = rio.open('Rasters/ksat.tif').profile
with rio.open('smca_results.tif', 'w', **profile) as dst:
    dst.write(result)
